In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import random
import numpy as np
import threading
import time

In [9]:
data = pd.read_csv("cloth/edges_ln.csv")
data.rename(columns={"balance" : "capacity", "fee_proportional" : "weight"}, inplace=True)
data = data[['from_node_id', 'to_node_id', 'capacity', 'weight']]

## preprocess
data = data.groupby(['from_node_id', 'to_node_id']).agg({'capacity': 'sum', 'weight': 'mean'}).reset_index()
data = data.astype({'weight' : int})
# data[(data['from_node_id'] == 1)]['capacity'].sum()
data

,from_node_id,to_node_id,capacity,weight
0,0,1,4300909,1
1,1,0,45699091,1
2,1,2,607791280,1
3,1,6,93361068,1
4,1,12,5393325728,1
...,...,...,...,...
54195,6002,345,19782854,1
54196,6002,854,187850564,1
54197,6003,5765,6738344,1
54198,6004,1931,235598450,1


In [14]:
nodeData = pd.read_csv("cloth/nodes_ln.csv")
nodeList = list(nodeData['id'])

In [17]:
G = nx.from_pandas_edgelist(data, 'from_node_id', 'to_node_id', edge_attr=['capacity', 'weight'], create_using=nx.DiGraph())
G.add_nodes_from(nodeList)

In [18]:
print([e for e in G.nodes()])

[0, 1, 2, 6, 12, 80, 89, 98, 100, 108, 128, 177, 194, 224, 251, 265, 282, 334, 353, 410, 507, 533, 600, 614, 619, 723, 724, 737, 745, 868, 907, 925, 1275, 1324, 1363, 1448, 1501, 1594, 1597, 1736, 1777, 1834, 1940, 1964, 1965, 1989, 2010, 2056, 2063, 2077, 2193, 2272, 2275, 2294, 2542, 2606, 2609, 2626, 2658, 3030, 3099, 3564, 3899, 3994, 4278, 4419, 5158, 5324, 5385, 5688, 5875, 3, 4, 5, 8, 10, 16, 17, 19, 21, 22, 24, 26, 27, 31, 34, 40, 41, 43, 48, 49, 52, 54, 58, 60, 62, 63, 64, 76, 78, 79, 86, 92, 94, 95, 101, 105, 106, 109, 111, 115, 119, 122, 130, 132, 139, 147, 150, 151, 153, 156, 160, 161, 162, 171, 184, 185, 186, 188, 189, 192, 197, 198, 199, 201, 202, 206, 211, 215, 216, 217, 220, 221, 222, 223, 227, 228, 231, 233, 236, 237, 238, 241, 243, 245, 247, 250, 252, 254, 256, 258, 263, 267, 275, 276, 278, 281, 287, 290, 292, 295, 296, 304, 306, 317, 319, 327, 330, 332, 335, 339, 342, 343, 344, 345, 346, 347, 348, 350, 351, 352, 354, 355, 356, 358, 359, 360, 362, 365, 368, 369, 370, 

In [74]:
%%time
sampleSize = 50
randS = random.sample(range(0, 6005), sampleSize)
randT = random.sample(range(0, 6005), sampleSize)
X = []
costCoef = 1
for i in range(sampleSize):
#     print(1)
    s, t = randS[i], randT[i]
    mincostFlow = nx.max_flow_min_cost(G, s, t)
#     print(2)
    mincost = nx.cost_of_flow(G, mincostFlow)
#     print(3)
    mincost /= 1000000
    mincostFlowValue = sum((mincostFlow[u][t] for u in G.predecessors(t))) - sum(
        (mincostFlow[t][v] for v in G.successors(t))
    )
#     print(4)
    
    fund = sum((G[s][u]['capacity'] for u in G.successors(s)))
#     print(5)
    if fund == 0:
        X.append(0)
        continue
    X.append((mincostFlowValue - costCoef * mincost) / fund)

print(X)

[0.9999966027187981, 0.9999969999999999, 0.9999830326663014, 0.9999728814263767, 0.00040543143695131214, 0.25773325834862826, 0.9999887215463623, 0.15043614519914836, 0.0, 0.00023303698123758063, 0.9999920290926725, 0.0, 0.9999160577632008, 0.9999971213342328, 0.9999968631315592, 0.38586175657938365, 0.00788094078321693, 0.0, 0, 0, 0.8009876893261272, 0.03473756370179422, 0, 0.9999131439423201, 0, 0, 0.0, 2.4856790779964982e-05, 0.8242446052097799, 0.9999969029115541, 0.9989979999999999, 0.0, 0.9999969048316552, 0.9999955915323737, 0.9997952337133765, 0.999996364560276, 0.9999872083128784, 0.9999885603687799, 0.017592255874015705, 0.05781290387738841, 0.9999977770237255, 0.0, 0.00022272324866334746, 0.999997, 0.20353161365691186, 2.2542309793833204e-05, 0.999939, 0.9999969999999999, 0.0, 0.03461212438147986]
CPU times: user 2min 20s, sys: 136 ms, total: 2min 20s
Wall time: 2min 20s


In [75]:
print("overall std :",np.std(X) / np.sqrt(sampleSize))
print("Lightning network EF :", np.mean(X))

overall std : 0.06731281406200168
Lightning network EF : 0.4954955688916348


In [73]:
dropCount = 10000
edgeList = list(G.edges)
r = random.sample(edgeList, dropCount)
G.remove_edges_from(r)

In [6]:
nodeData = pd.read_csv("cloth/nodes_ln.csv")
l = list(nodeData['id'])
# print(l)

In [33]:
import pandas as pd

def remove_specific_row_from_csv(file, args):
    df = pd.read_csv(file)
    df = df.drop(index=args)
    df.to_csv(file, index=False)



In [35]:
remove_specific_row_from_csv("./cloth/payments.csv", [0, 1, 2])

In [14]:
#betweenness
# bc = nx.betweenness_centrality(G, weight=False, normalized=False)
# bc

In [15]:
#closeness

cc = nx.closeness_centrality(G)

In [18]:
#degree
dc = nx.degree_centrality(G)

In [19]:
evc = nx.eigenvector_centrality(G)

In [17]:
dataSize = 100
randNodes = random.sample(range(0, 6005), dataSize)


0.3377045392093966

In [12]:
payments = pd.read_csv("./cloth/output_dir/payments_output.csv")
succeededCount = payments[(payments['sender_id'] == 1710) & (payments['is_success'] == 1)].count()[0]
succeededCount

2